# mwb_flow.prep Package Example
This notebook provides a complete example of acquiring meterology data and combining it with stream gage data to produce a formatted input data file for running the monthly water balance flow (mwb_flow) model (Gray and McCabe, 2010).
A subset of dates were used (1/1/2016 - 12/31/2016) to reduce the time spent getting gridmet data while testing.

First import necessary packages/modules

In [ ]:
import os
from pathlib import Path
import geopandas as gpd
import xarray as xr



mwb_flow_dir = r'C:\Users\CND905\Downloaded_Programs\mwb_flow'
os.chdir(mwb_flow_dir)

from prep.datafile import CreateInputFile
from prep.metdata import get_gridmet_at_points
from prep.datafile import check_format

Import a shape file with deliniated watershed polygon. This shape file has an attribute table with a column used to index the geometries. In this case, a column gage station numbers was used since the numbers will later be used to merge meterologic data with streamflow data.

In [ ]:
exres_pth = Path(r'C:\Users\CND905\Downloaded_Programs\mwb_flow\Examples\data\Lolo_WB_Model_Calibration_Catchments_32611.shp')
exres = gpd.read_file(exres_pth)
# This file is in crs 32611 (WGS84 UTM zone 11N), need it to be 4326 for getting GridMET.
exres = exres.to_crs(4326)

In [ ]:
# This example provides a unique identifier column rather than using the default DataFrame index.
# TODO change the gdf_index_col to accept full gage ID numbers?
exres_met = get_gridmet_at_points(exres, 'gageID', start='2016-01-01', end='2016-12-31')

A object with meterology data was made using the CreateInputFile class. This class initialization and methods have been edited from the pydlem.prep module which required lake parameters inputs. This class will be further updated to create a method for merg stream gage data. Note that the data frequency is daily and needs to be reduced to monthly. 
This cell is here to demonstrate/test if the updated CreateInputFile code is working for now.  

In [ ]:
test = CreateInputFile(geoms=None, met_data=exres_met, met_source='from_file')
test.data

The daily temperature and precipitation data are summarised to monthly frequencies below. The resample function allows for simple grouping and the application of mathamatical functions. However, doing so changes the dimentions of an array, which makes this tricky. Perhaps a diffrent workflow is needed. 

Here, the daily temperature is averaged to calculate the monthly temperature. 

In [ ]:
# Calculate daily mean air temperature then followed by the monthly mean air temperature
mean_temp = ((exres_met.min_temp + exres_met.max_temp) / 2) - 273.15  # also convert to Celcius from GridMET native Kelvin
mean_temp
monthly_temp = mean_temp.resample(time = "MS").mean()

# Convert to a DataArray with attributes and title
Monthly_Temp = xr.DataArray(monthly_temp, coords=monthly_temp.coords, attrs={'standard_name': 'Monthly_Temperature', 'units': 'Celcius'})
Monthly_Temp.name = 'mo_temp'
Monthly_Temp

And the daily precipitation summed to calcualte the montly precipitation. 

In [ ]:
# Calculate the monthly precipitation
monthly_precip = exres_met.precip.resample(time = "MS").sum()

# Convert to a DataArray with attributes and title
Monthly_Precip = xr.DataArray(monthly_precip, coords=monthly_precip.coords, attrs={'standard_name': 'Monthy Precipitation', 'units': 'mm'})
Monthly_Precip.name = 'mo_precip'
Monthly_Precip

A DataArray of the monthly discharge is imported from the mwb_flow.q_datafile modual. This data was summarized by month, labeled with stream gage station number identifiers, and concatinated. T
The dimentions are similar to the mo_temp and mo_precip DataArrays but the locations are in a diffrent order. The code in the mwb_flow.q_datafile modual can be cleaned up and formated as a class with methods for simpilar use. This example could be a "from file" argument when creating an object. 

Merging each varaible into a DataSet currently results in incorrect coordinates for the data. 

In [ ]:
Monthly_Q = xr.load_dataarray(r'C:\Users\CND905\Downloaded_Programs\mwb_flow\prep\q_datafile_output.nc')
Monthly_Q

xr.combine_by_coords can merg the Monthly_Temp and Montly_Precip DataArrays but has trouble merging with Montly_Q DataArray. Here, the dimentions increase from 6 locations to 12. Need to troubleshoot this to make sure the merging is reliable if using this workflow. 

In [ ]:
combined = xr.combine_by_coords(data_objects=[Monthly_Temp, Monthly_Precip, Monthly_Q])
combined
